# MS2DeepScore

In [2]:
!pip install matchms ms2deepscore tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.0/205.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 134.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 138.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully unin

In [1]:
import os
import numpy as np

from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from matchms.filtering import (
    normalize_intensities,
    select_by_mz,
    reduce_to_number_of_peaks,
    require_minimum_number_of_peaks,
)
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from ms2deepscore import MS2DeepScore
from matchms import calculate_scores

In [3]:
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
import os

# Input MGF files
mgf_files = [
    "negative_training_spectra.mgf",
    "negative_validation_spectra.mgf",
    "negative_testing_spectra.mgf",
    "positive_training_spectra.mgf",
    "positive_validation_spectra.mgf",
    "positive_testing_spectra.mgf"
]

# Output directory
out_dir = "mgf_1000"
os.makedirs(out_dir, exist_ok=True)

MAX_SPECTRA = 1000

for mgf_file in mgf_files:
    print(f"\nProcessing {mgf_file}")

    # Load spectra
    spectra = list(load_from_mgf(mgf_file))
    print(f"  Total spectra in file: {len(spectra)}")

    # Take first 1000
    spectra_1000 = spectra[:MAX_SPECTRA]
    print(f"  Keeping: {len(spectra_1000)}")

    # Output filename
    base = os.path.basename(mgf_file)
    out_file = os.path.join(out_dir, base.replace(".mgf", "_1000.mgf"))

    # Save
    save_as_mgf(spectra_1000, out_file)
    print(f"  Saved to: {out_file}")



Processing negative_training_spectra.mgf
  Total spectra in file: 130901
  Keeping: 1000
  Saved to: mgf_1000/negative_training_spectra_1000.mgf

Processing negative_validation_spectra.mgf
  Total spectra in file: 7551
  Keeping: 1000
  Saved to: mgf_1000/negative_validation_spectra_1000.mgf

Processing negative_testing_spectra.mgf
  Total spectra in file: 7142
  Keeping: 1000
  Saved to: mgf_1000/negative_testing_spectra_1000.mgf

Processing positive_training_spectra.mgf
  Total spectra in file: 469257
  Keeping: 1000
  Saved to: mgf_1000/positive_training_spectra_1000.mgf

Processing positive_validation_spectra.mgf
  Total spectra in file: 25412
  Keeping: 1000
  Saved to: mgf_1000/positive_validation_spectra_1000.mgf

Processing positive_testing_spectra.mgf
  Total spectra in file: 24911
  Keeping: 1000
  Saved to: mgf_1000/positive_testing_spectra_1000.mgf


# Everything Below Was Not Used in the Final Product

In [ ]:
original_mgf = "ms2deepscore_data/positive_training_spectra.mgf"
spectra = list(load_from_mgf(original_mgf))
print(len(spectra))

469257


In [ ]:
import os
os.makedirs("ms2deepscore_small", exist_ok=True)

In [ ]:
from matchms.exporting import save_as_mgf
save_as_mgf(spectra, "ms2deepscore_small/small_training_valid.mgf")

In [ ]:
fixed_spectra = []

for s in spectra:
    if s is None:
        continue

    meta = s.metadata
    if "inchikey" not in meta:
        if "inchi" in meta:
            mol = Chem.MolFromInchi(meta["inchi"])
            if mol:
                meta["inchikey"] = Chem.inchi.MolToInchiKey(mol)
            else:
                continue
        else:
            continue
    if "precursor_mz" not in meta:
        if "parent_mass" in meta:
            meta["precursor_mz"] = meta["parent_mass"]
        else:
            continue
    if "ionmode" not in meta:
        continue
    if meta["ionmode"].lower() != "positive":
        continue
    fixed_spectra.append(s)
print(len(fixed_spectra))

469257


In [ ]:
def fix_metadata(s):
    if "parent_mass" in s.metadata:
        try:
            s.metadata["parent_mass"] = float(s.metadata["parent_mass"])
        except:
            del s.metadata["parent_mass"]
    if "precursor_mz" in s.metadata:
        try:
            s.metadata["precursor_mz"] = float(s.metadata["precursor_mz"])
        except:
            del s.metadata["precursor_mz"]
    return s
fixed_spectra = [fix_metadata(s) for s in fixed_spectra if s is not None]

In [ ]:
#from rdkit import Chem
#from rdkit.Chem import inchi

#def ensure_inchikey(s):
#    if "inchikey" in s.metadata and s.metadata["inchikey"].startswith("InChI="):
#        mol = Chem.MolFromInchi(s.metadata["inchikey"])
#        if mol:
#            s.metadata["inchikey"] = inchi.MolToInchiKey(mol)
#    return s
#spectra = [ensure_inchikey(s) for s in spectra]

In [ ]:
seen = set()
unique_spectra = []

for s in fixed_spectra:
    ik = s.metadata["inchikey"]
    if ik not in seen:
        unique_spectra.append(s)
        seen.add(ik)

print(f"Unique InChIKeys: {len(unique_spectra)}")

Unique InChIKeys: 36852


In [ ]:
os.makedirs("ms2deepscore_small", exist_ok=True)
small_mgf_file = "ms2deepscore_small/small_training_valid.mgf"
save_as_mgf(unique_spectra, small_mgf_file)
print("Saved:", small_mgf_file)


Saved: ms2deepscore_small/small_training_valid.mgf


In [ ]:
check = list(load_from_mgf(small_mgf_file))
print("Reloaded spectra:", len(check))
print("Example metadata:")
print(check[0].metadata)


Reloaded spectra: 36852
Example metadata:
{'charge': 1, 'description': 'Enamine and Molport', 'formula': 'C7H5N3O2', 'inchi': 'InChI=1S/C7H5N3O2/c11-7(12)5-6-8-2-1-3-10(6)4-9-5/h1-4H,(H,11,12)', 'feature_id': '339', 'adduct': '[M-H2O+H]+', 'feature_ms1_height': '1.279E6', 'collision_energy': '60.0', 'fragmentation_method': 'HCD', 'isolation_window': '1.200000047684', 'acquisition': 'Crude', 'instrument_type': 'Orbitrap', 'ims_type': 'none', 'ion_source': 'ESI', 'ionmode': 'positive', 'dataset_id': 'MSVPLACEHOLDERID', 'usi': 'mzspec:MSVPLACEHOLDERID:20240405_pluskal_enamine_5002_A4_id_MSn_positive_2uL.mzML:339', 'scans': '339', 'precursor_purity': '0.37398476724755064', 'quality_chimeric': 'CHIMERIC', 'quality_explained_intensity': '0.61782515', 'quality_explained_signals': '0.48387095', 'num_peaks': '31', 'compound_name': 'imidazo[1,5-a]pyrimidine-8-carboxylic acid (Chimeric precursor selection)', 'parent_mass': '163.03818', 'inchi_aux': 'WXRSEUXBXNDCKE-UHFFFAOYSA-N', 'ms_level': '2', 

In [ ]:
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
import numpy as np
import os

input_mgf = "ms2deepscore_data/positive_training_spectra.mgf"
output_mgf = "ms2deepscore_small/small_training_valid.mgf"
os.makedirs("ms2deepscore_small", exist_ok=True)

max_spectra = 1000

all_spectra = []
count = 0
for s in load_from_mgf(input_mgf):
    inchi = s.metadata.get("INCHI") or s.metadata.get("inchikey")
    mz = s.metadata.get("precursor_mz")
    ion = s.metadata.get("ionmode")
    if inchi and mz and ion == "positive":
        all_spectra.append(s)
        count += 1
    if count >= max_spectra:
        break

print(f"Total spectra saved for small MGF: {len(all_spectra)}")
save_as_mgf(all_spectra, output_mgf)


Total spectra saved for small MGF: 1000


In [ ]:
import requests
import os
from tqdm import tqdm

def download_file(link, file_name):
    response = requests.get(link, stream=True)
    if os.path.exists(file_name):
        print(f"The file {file_name} already exists, the file won't be downloaded")
        return
    total_size = int(response.headers.get('content-length', 0))

    with open(file_name, "wb") as f, tqdm(desc="Downloading file", total=total_size, unit='B', unit_scale=True, unit_divisor=1024,) as bar:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
model_file_name = "ms2deepscore_model.pt"
download_file("https://zenodo.org/records/14290920/files/settings.json?download=1", "ms2deepscore_settings.json")
download_file("https://zenodo.org/records/14290920/files/ms2deepscore_model.pt?download=1", model_file_name)

In [ ]:
from ms2deepscore.models import load_model
from matchms.Pipeline import Pipeline, create_workflow
from matchms.filtering.default_pipelines import DEFAULT_FILTERS
from ms2deepscore import MS2DeepScore
from matchms.importing import load_from_mgf

model_file_name = "ms2deepscore_model.pt"
model = load_model(model_file_name, allow_legacy=True)

spectra_file_name = "ms2deepscore_small/small_training_valid.mgf"
spectra = list(load_from_mgf(spectra_file_name))

pipeline = Pipeline(
    create_workflow(
        query_filters=DEFAULT_FILTERS,
        score_computations=[[MS2DeepScore, {"model": model}]]
    )
)

report = pipeline.run(spectra_file_name)
similarity_matrix = pipeline.scores.to_array()

ms2ds_model = MS2DeepScore(model)
embeddings = ms2ds_model.get_embedding_array(report.spectra_queries)


/usr/local/lib/python3.12/dist-packages/ms2deepscore/models/load_model.py:151: RuntimeWarning: Using UNSAFE legacy loading (weights_only=False). Only do this for trusted files.
  warnings.warn(


2025-12-15 17:56:55,105:WARNING:matchms:Pipeline:The order of the filters has been changed compared to the Yaml file.


2025-12-15 17:56:55,109:WARNING:matchms:Pipeline:The order of the filters has been changed compared to the Yaml file.


2025-12-15 17:56:55,113:WARNING:matchms:Pipeline:No logging file was defined.Logging messages will not be written to file.


Processing spectra: 24it [00:00, 76.23it/s]

...

Processing spectra: 37025it [10:34, 71.48it/s]

2025-12-15 18:07:29,690:WARNING:matchms:clean_adduct:The charge in the adduct: [M-H2O+H]+ and the given charge: 2 do not match


2025-12-15 18:07:29,712:WARNING:matchms:clean_adduct:The charge in the adduct: [M-H2O+H]+ and the given charge: 2 do not match


2025-12-15 18:07:29,732:WARNING:matchms:clean_adduct:The charge in the adduct: [M-H2O+H]+ and the given charge: 2 do not match


Processing spectra: 37852it [10:48, 58.41it/s]
Computing spectral embeddings ...: 100%|██████████| 37852/37852 [21:52<00:00, 28.83it/s]


In [7]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
import numpy as np
from ms2deepscore.models import load_model
from matchms.Pipeline import Pipeline, create_workflow
from matchms.filtering.default_pipelines import DEFAULT_FILTERS
from ms2deepscore import MS2DeepScore
from matchms.importing import load_from_mgf

model_file_name = "ms2deepscore_model.pt"
model = load_model(model_file_name, allow_legacy=True)

pipeline = Pipeline(
    create_workflow(
        query_filters=DEFAULT_FILTERS,
        score_computations=[[MS2DeepScore, {"model": model}]]
    )
)

report = pipeline.run("ms2deepscore_small/small_training_valid.mgf")
similarity_matrix = pipeline.scores.to_array()

ms2ds_model = MS2DeepScore(model)
embeddings = ms2ds_model.get_embedding_array(report.spectra_queries)

def compute_tanimoto_similarity(spectra):
    fps = []
    for s in spectra:
        inchi_str = s.metadata.get("inchikey") or s.metadata.get("INCHI")
        if inchi_str:
            mol = Chem.MolFromInchi(inchi_str)
            if mol:
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
                fps.append(fp)
            else:
                fps.append(None)
        else:
            fps.append(None)

    n = len(fps)
    tanimoto_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if fps[i] is not None and fps[j] is not None:
                tanimoto_matrix[i, j] = DataStructs.TanimotoSimilarity(fps[i], fps[j])
            else:
                tanimoto_matrix[i, j] = np.nan
    return tanimoto_matrix


tanimoto_matrix = compute_tanimoto_similarity(report.spectra_queries)
mask = ~np.isnan(tanimoto_matrix)
rmse = np.sqrt(np.mean((similarity_matrix[mask] - tanimoto_matrix[mask])**2))
print("RMSE between MS2DeepScore and Tanimoto similarities:", rmse)

2025-12-15 18:38:53,263:WARNING:matchms:Pipeline:The order of the filters has been changed compared to the Yaml file.


2025-12-15 18:38:53,268:WARNING:matchms:Pipeline:The order of the filters has been changed compared to the Yaml file.


2025-12-15 18:38:53,290:WARNING:matchms:Pipeline:No logging file was defined.Logging messages will not be written to file.


Processing spectra: 20it [00:00, 69.39it/s]

2025-12-15 18:38:53,683:WARNING:matchms:clean_adduct:The charge in the adduct: [M-H2O+H]+ and the given charge: 2 do not match


...

2025-12-15 18:39:02,656:WARNING:matchms:clean_adduct:The charge in the adduct: [M+H]+ and the given charge: 2 do not match


Processing spectra: 1366it [00:21, 46.18it/s]

2025-12-15 18:39:14,420:WARNING:matchms:clean_adduct:The charge in the adduct: [M+H]+ and the given charge: 2 do not match


Processing spectra: 1626it [00:26, 62.08it/s]


KeyboardInterrupt: 